# Model-Based метод кинетического анализа

Model-based метод — это подход к определению кинетических параметров твердофазных реакций, основанный на решении системы обыкновенных дифференциальных уравнений (ОДУ), описывающих кинетику многостадийных реакций.

В отличие от model-free методов (Фридмана, Киссинджера, Вязовкина), model-based подход предполагает явную форму кинетической модели и позволяет одновременно оптимизировать параметры нескольких последовательных или параллельных реакций.

---

## Концентрация vs конверсия

В данном модуле используется **концентрация** $e$ (доля непрореагировавшего вещества) вместо конверсии $\alpha$:

$$ e = 1 - \alpha \tag{1}$$

$$ \alpha = 1 - e \tag{2}$$

где:
- $e$ — концентрация (доля непрореагировавшего материала), изменяется от 1 до 0
- $\alpha$ — степень превращения (доля прореагировавшего материала), изменяется от 0 до 1

Все кинетические модели в `NUC_MODELS_TABLE` определены как функции от $e$.

---

## Уравнение Аррениуса

Температурная зависимость константы скорости реакции описывается уравнением Аррениуса:

$$ k(T) = A \cdot \exp\left(-\frac{E_a}{RT}\right) \tag{3}$$

где:
- $k(T)$ — константа скорости реакции при температуре $T$, с⁻¹
- $A$ — предэкспоненциальный множитель (частотный фактор), с⁻¹
- $E_a$ — энергия активации, кДж/моль (в коде используется Дж/моль)
- $R$ — универсальная газовая постоянная = 8.314 Дж/(моль·К)
- $T$ — абсолютная температура, К

В коде используется $\log_{10}(A)$ вместо $A$ для удобства оптимизации:

$$ k(T) = 10^{\log A} \cdot \exp\left(-\frac{E_a \cdot 1000}{RT}\right) \tag{4}$$

---

## ОДУ система для многостадийных реакций

Для схемы реакций с $N$ компонентами и $M$ реакциями:

$$ \text{Компоненты: } C_1, C_2, \ldots, C_N $$
$$ \text{Реакции: } C_i \xrightarrow{k_m} C_j $$

Система ОДУ для концентраций $Y_k$ и скоростей реакций $r_m$:

$$ \frac{dY_k}{dT} = -\sum_{\text{реагенты}} r_m + \sum_{\text{продукты}} r_m \tag{5}$$

где скорость каждой реакции:

$$ r_m = k_m(T) \cdot f_m(e_{\text{source}}) \cdot \text{contribution}_m \tag{6}$$

$f_m(e)$ — дифференциальная форма кинетической модели (F2, A2, R3 и т.д.)

**Пример для схемы A → B → C → D:**

$$ \frac{d[A]}{dT} = -k_1 \cdot f_1([A]) $$
$$ \frac{d[B]}{dT} = k_1 \cdot f_1([A]) - k_2 \cdot f_2([B]) $$
$$ \frac{d[C]}{dT} = k_2 \cdot f_2([B]) - k_3 \cdot f_3([C]) $$
$$ \frac{d[D]}{dT} = k_3 \cdot f_3([C]) $$

---

## Метод интеграции BDF

Для решения системы ОДУ используется метод **BDF (Backward Differentiation Formula)**:

$$ \texttt{solve\_ivp(ode\_func, [T\_start, T\_end], y\_0, method="BDF")} $$

**Почему BDF, а не RK45?**

BDF — это неявный многошаговый метод, предпочтительный для **жёстких (stiff) систем**:

- Жёсткие системы возникают, когда реакции имеют сильно различные константы скорости ($k_1 \gg k_2$ или $E_{a1} \gg E_{a2}$)
- Явные методы (RK45) требуют очень малого шага для устойчивости
- BDF позволяет использовать большие шаги без потери устойчивости

**Timeout:** Каждое интегрирование ограничено 50 мс через декоратор `@integration_timeout(50.0)` для предотвращения зависания при некорректных параметрах.

---

## Оптимизация differential_evolution

Для минимизации MSE между моделью и экспериментом используется **дифференциальная эволюция** (DE) из SciPy:

$$ \text{MSE}_{\text{total}} = \sum_{\beta} \text{mean}\left((M_{\text{exp}}^{(\beta)} - M_{\text{model}}^{(\beta)})^2\right) \tag{7}$$

**Параметры оптимизации:**

| Параметр       | Значение  | Описание                           |
|----------------|-----------|-------------------------------------|
| `strategy`     | best1bin  | Стратегия мутации                   |
| `maxiter`      | 1000      | Макс. число итераций                |
| `popsize`      | 50        | Размер популяции                    |
| `tol`          | 0.01      | Допуск сходимости                   |
| `mutation`     | (0.4, 1.2)| Диапазон мутации                    |
| `recombination`| 0.7       | Вероятность кроссовера              |
| `init`         | latinhypercube | Начальная инициализация        |
| `workers`      | 6         | Параллельные воркеры                |

**Вектор параметров** для $M$ реакций:

$$ \text{params} = [\log A_1, \ldots, \log A_M, E_{a1}, \ldots, E_{aM}, \text{model}_1, \ldots, \text{model}_M, \text{contrib}_1, \ldots, \text{contrib}_M] $$

---



---

## Загрузка данных и схема реакций

Для демонстрации model-based метода используем экспериментальные данные TGA (термогравиметрический анализ) соединения NH₄NO₃ при трёх скоростях нагрева.

### Схема реакций

Многостадийная схема разложения:

$$ A \xrightarrow{k_1} B \xrightarrow{k_2} C \xrightarrow{k_3} D $$

где:
- **A** — исходное вещество (NH₄NO₃)
- **B, C** — промежуточные продукты
- **D** — конечный продукт

### Начальные параметры реакций (из логов)

| Реакция | $\log_{10}(A)$ | $E_a$ (кДж/моль) | Модель | Вклад |
|---------|----------------|-------------------|--------|-------|
| A → B   | 8.0            | 120.0             | F2     | 0.5   |
| B → C   | 8.0            | 120.0             | F2     | 0.5   |
| C → D   | 8.0            | 120.0             | F2     | 0.5   |

### Границы оптимизации (bounds)

| Параметр        | Min    | Max   |
|-----------------|--------|-------|
| $E_a$ (кДж/моль) | 30.0   | 250.0 |
| $\log_{10}(A)$  | -15.0  | 30.0  |
| Contribution    | 0.01   | 1.0   |

### Разрешённые модели для перебора

```python
allowed_models = ["A2/3", "A3", "A3/2", "F1/3", "F1/A1", "F2", "F3", "G1", "G2", "R2", "R3"]
```

**Примечание:** Вклады (contributions) для последовательной цепи реакций не обязательно должны суммироваться в 1 — они определяют вклад каждой реакции в суммарную потерю массы.

In [1]:
# %% [code]
import numpy as np
import pandas as pd
from pathlib import Path

# === ЗАГРУЗКА ДАННЫХ (как в логах приложения) ===
# Файлы загружаются отдельно для каждой скорости нагрева β
PROJECT_ROOT = Path("c:/IDE/repository/solid-state_kinetics")

# Скорости нагрева β (K/min)
betas = [3.0, 5.0, 10.0]

# Загрузка отдельных файлов
# CSV содержит заголовок в первой строке: temperature,rate_X
file_paths = [
    PROJECT_ROOT / "resources/NH4_rate_3.csv",
    PROJECT_ROOT / "resources/NH4_rate_5.csv",
    PROJECT_ROOT / "resources/NH4_rate_10.csv"
]

dataframes = []
for path in file_paths:
    print(f"Загрузка: {path.name}")
    df_temp = pd.read_csv(path, delimiter=",")
    print(f"  Строк: {len(df_temp)}, Столбцы: {list(df_temp.columns)}")
    dataframes.append(df_temp)

# Объединение данных по температуре
# Извлекаем температуру (одинаковая во всех файлах)
exp_temperature = dataframes[0]["temperature"].values + 273.15  # K

# Извлекаем массы (rate_X — это масса в %, нормализуем к [0, 1])
exp_mass_3 = dataframes[0]["rate_3"].values / 100.0
exp_mass_5 = dataframes[1]["rate_5"].values / 100.0
exp_mass_10 = dataframes[2]["rate_10"].values / 100.0
all_exp_masses = [exp_mass_3, exp_mass_5, exp_mass_10]

print(f"\nДиапазон температур: {exp_temperature[0]:.1f} K ... {exp_temperature[-1]:.1f} K")
print(f"Диапазон масс β=3: {exp_mass_3[0]:.3f} ... {exp_mass_3[-1]:.3f}")

# === СХЕМА РЕАКЦИЙ ===
scheme = {
    "components": [
        {"id": "A"},
        {"id": "B"},
        {"id": "C"},
        {"id": "D"}
    ],
    "reactions": [
        {
            "from": "A", "to": "B",
            "reaction_type": "F2",
            "Ea": 120.0,           # кДж/моль
            "log_A": 8.0,          # log₁₀(A), A в с⁻¹
            "contribution": 0.5,
            "Ea_min": 30.0, "Ea_max": 200.0,
            "log_A_min": -15.0, "log_A_max": 30.0,
            "contribution_min": 0.01, "contribution_max": 1.0,
            "allowed_models": ["A2/3", "A3", "A3/2", "F1/3", "F1/A1", "F2", "F3", "G1", "G2", "R2", "R3"]
        },
        {
            "from": "B", "to": "C",
            "reaction_type": "F2",
            "Ea": 120.0,
            "log_A": 8.0,
            "contribution": 0.5,
            "Ea_min": 30.0, "Ea_max": 250.0,
            "log_A_min": -15.0, "log_A_max": 30.0,
            "contribution_min": 0.01, "contribution_max": 1.0,
            "allowed_models": ["A2/3", "A3", "A3/2", "F1/3", "F1/A1", "F2", "F3", "G1", "G2", "R2", "R3"]
        },
        {
            "from": "C", "to": "D",
            "reaction_type": "F2",
            "Ea": 120.0,
            "log_A": 8.0,
            "contribution": 0.5,
            "Ea_min": 30.0, "Ea_max": 250.0,
            "log_A_min": -15.0, "log_A_max": 30.0,
            "contribution_min": 0.01, "contribution_max": 1.0,
            "allowed_models": ["A2/3", "A3", "A3/2", "F1/3", "F1/A1", "F2", "F3", "G1", "G2", "R2", "R3"]
        }
    ]
}

# Вывод схемы реакций
print("\n=== Схема реакций ===")
for rxn in scheme["reactions"]:
    print(f"  {rxn['from']} → {rxn['to']}: logA={rxn['log_A']}, Ea={rxn['Ea']} kJ/mol, model={rxn['reaction_type']}, contrib={rxn['contribution']}")

# === ПАРАМЕТРЫ ОПТИМИЗАЦИИ ===
de_params = {
    "strategy": "best1bin",
    "maxiter": 1000,
    "popsize": 50,
    "tol": 0.01,
    "mutation": (0.4, 1.2),
    "recombination": 0.7,
    "init": "latinhypercube",
    "workers": 6,
    "polish": False
}

print("\n=== Параметры DE ===")
for k, v in de_params.items():
    print(f"  {k}: {v}")

# Производные переменные для расчёта
species_list = [c["id"] for c in scheme["components"]]
reactions = scheme["reactions"]
num_species = len(species_list)
num_reactions = len(reactions)

print(f"\nЧисло компонентов: {num_species}")
print(f"Число реакций: {num_reactions}")

Загрузка: NH4_rate_3.csv
  Строк: 490, Столбцы: ['temperature', 'rate_3']
Загрузка: NH4_rate_5.csv
  Строк: 490, Столбцы: ['temperature', 'rate_5']
Загрузка: NH4_rate_10.csv
  Строк: 490, Столбцы: ['temperature', 'rate_10']

Диапазон температур: 304.4 K ... 771.3 K
Диапазон масс β=3: 0.995 ... -0.004

=== Схема реакций ===
  A → B: logA=8.0, Ea=120.0 kJ/mol, model=F2, contrib=0.5
  B → C: logA=8.0, Ea=120.0 kJ/mol, model=F2, contrib=0.5
  C → D: logA=8.0, Ea=120.0 kJ/mol, model=F2, contrib=0.5

=== Параметры DE ===
  strategy: best1bin
  maxiter: 1000
  popsize: 50
  tol: 0.01
  mutation: (0.4, 1.2)
  recombination: 0.7
  init: latinhypercube
  workers: 6
  polish: False

Число компонентов: 4
Число реакций: 3


---

## ОДУ интеграция

### Кинетические модели

Таблица `NUC_MODELS_TABLE` содержит дифференциальные формы кинетических моделей:

| Модель | $f(e)$ | Описание |
|--------|--------|----------|
| F1/3 | $\frac{3}{2}e^{1/3}$ | Порядок 1/3 |
| F2 | $e^2$ | Второй порядок |
| F3 | $e^3$ | Третий порядок |
| A2 | $2e[-\ln(e)]^{1/2}$ | Аврами-Erofeev 2 |
| A3 | $3e[-\ln(e)]^{2/3}$ | Аврами-Erofeev 3 |

### Timeout декоратор

Для предотвращения зависания при некорректных параметрах используется декоратор `@integration_timeout(50.0)` — каждое интегрирование ограничено 50 мс.

### Функция `integrate_ode_for_beta`

Выполняет интегрирование ОДУ для одной скорости нагрева β:

```python
sol = solve_ivp(ode_wrapper, [T_start, T_end], y0, t_eval=exp_temperature, method="BDF")
```

Возвращает:
- `sol.success` — флаг успешности
- `sol.y` — матрица решений (концентрации + скорости реакций)

In [2]:
# %% [code]
import threading
import warnings
from functools import wraps
from scipy.integrate import solve_ivp

# Подавление RuntimeWarning от scipy BDF solver (жёсткие системы)
warnings.filterwarnings("ignore", category=RuntimeWarning, module="scipy.integrate")

# === КОНСТАНТЫ И ТАБЛИЦА МОДЕЛЕЙ ===
R = 8.314  # Универсальная газовая постоянная, Дж/(моль·К)
EPS = 1e-10  # Малое число для защиты от log(0)

def safe_e(e):
    """Защита концентрации от граничных значений (e=0 вызывает log(0))."""
    return np.clip(e, EPS, 1.0 - EPS)

NUC_MODELS_TABLE = {
    "F1/3": {"differential_form": lambda e: (3 / 2) * np.clip(e, EPS, None) ** (1 / 3)},
    "F3/4": {"differential_form": lambda e: 4 * np.clip(e, EPS, None) ** (3 / 4)},
    "F3/2": {"differential_form": lambda e: 2 * np.clip(e, EPS, None) ** (3 / 2)},
    "F2":   {"differential_form": lambda e: e ** 2},
    "F3":   {"differential_form": lambda e: e ** 3},
    "F1/A1": {"differential_form": lambda e: e},
    "A2":   {"differential_form": lambda e: 2 * e * (-np.log(safe_e(e))) ** (1 / 2)},
    "A3":   {"differential_form": lambda e: 3 * e * (-np.log(safe_e(e))) ** (2 / 3)},
    "A2/3": {"differential_form": lambda e: 2 * e * (-np.log(safe_e(e))) ** (1 / 3)},
    "G1":   {"differential_form": lambda e: np.clip(e, EPS, None) ** (1 / 2)},
    "G2":   {"differential_form": lambda e: np.clip(e, EPS, None) ** (1 / 3)},
    "R2":   {"differential_form": lambda e: 2 * np.clip(e, EPS, None) ** (1 / 2)},
    "R3":   {"differential_form": lambda e: 3 * np.clip(e, EPS, None) ** (2 / 3)},
}

# === TIMEOUT ДЕКОРАТОР ===
class TimeoutError(Exception):
    """Исключение при превышении времени интегрирования."""
    pass

def integration_timeout(timeout_ms: float):
    """Декоратор для ограничения времени выполнения функции."""
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            result = [None]
            exception = [None]
            
            def target():
                try:
                    result[0] = func(*args, **kwargs)
                except Exception as e:
                    exception[0] = e
            
            thread = threading.Thread(target=target, daemon=True)
            thread.start()
            thread.join(timeout_ms / 1000.0)
            
            if thread.is_alive():
                raise TimeoutError(f"Integration timeout after {timeout_ms}ms")
            if exception[0] is not None:
                raise exception[0]
            return result[0]
        return wrapper
    return decorator

# === ОДУ ФУНКЦИЯ ===
def ode_function(T, y, beta, params, species_list, reactions, num_species, num_reactions):
    """
    Правая часть системы ОДУ для кинетики реакций.
    
    Args:
        T: температура, K
        y: вектор состояния [концентрации..., скорости реакций...]
        beta: скорость нагрева, K/min
        params: вектор параметров [logA..., Ea..., model_idx..., contrib...]
        species_list: список компонентов ['A', 'B', 'C', 'D']
        reactions: список реакций с параметрами
        num_species: число компонентов
        num_reactions: число реакций
    
    Returns:
        dYdt: производные по температуре
    """
    dYdt = np.zeros_like(y)
    
    for i, rxn in enumerate(reactions):
        src = rxn["from"]
        tgt = rxn["to"]
        src_index = species_list.index(src)
        tgt_index = species_list.index(tgt)
        
        # Концентрация реагента (защита от отрицательных значений)
        e_value = max(y[src_index], 0.0)
        
        # Извлечение параметров из вектора
        logA = params[i]
        Ea = params[num_reactions + i]  # кДж/моль
        
        # Выбор модели
        model_param_index = 2 * num_reactions + i
        allowed_models = rxn["allowed_models"]
        model_index = int(np.clip(round(params[model_param_index]), 0, len(allowed_models) - 1))
        model_key = allowed_models[model_index]
        
        # Дифференциальная форма модели
        model = NUC_MODELS_TABLE.get(model_key)
        f_e = model["differential_form"](e_value) if model else e_value
        
        # Защита от nan/inf
        if not np.isfinite(f_e):
            f_e = 0.0
        
        # Константа скорости: k = A * exp(-Ea*1000 / (R*T)) / beta
        # beta в K/min, переводим в K/s: beta_SI = beta / 60
        beta_SI = beta / 60.0
        k_i = (10**logA * np.exp(-Ea * 1000 / (R * T))) / beta_SI
        
        # Скорость реакции
        rate = k_i * f_e
        
        # Обновление производных концентраций
        dYdt[src_index] -= rate
        dYdt[tgt_index] += rate
        
        # Сохранение мгновенной скорости реакции
        dYdt[num_species + i] = rate
    
    return dYdt

# === ИНТЕГРАЦИЯ С TIMEOUT ===
@integration_timeout(50.0)
def integrate_ode_for_beta(beta, params, species_list, reactions, num_species, num_reactions, 
                           exp_temperature, exp_mass):
    """
    Интегрирование ОДУ для одной скорости нагрева β.
    
    Returns:
        mse: среднеквадратичная ошибка между моделью и экспериментом
        или 1e4 при ошибке/timeout
    """
    # Начальные условия: [1, 0, 0, 0, 0, 0, 0] (A=1, остальные 0)
    y0 = np.zeros(num_species + num_reactions)
    y0[0] = 1.0
    
    def ode_wrapper(T, y):
        return ode_function(T, y, beta, params, species_list, reactions, num_species, num_reactions)
    
    # Интегрирование методом BDF (для жёстких систем)
    sol = solve_ivp(
        ode_wrapper, 
        [exp_temperature[0], exp_temperature[-1]], 
        y0,
        t_eval=exp_temperature,
        method="BDF"
    )
    
    if not sol.success:
        return 1e4, None
    
    # Интегрированные скорости реакций (накопленная конверсия)
    rates_int = sol.y[num_species : num_species + num_reactions, :]
    
    # Contributions из вектора параметров
    contributions = params[3 * num_reactions : 4 * num_reactions]
    
    # Суммарная конверсия
    int_sum = np.sum(contributions[:, np.newaxis] * rates_int, axis=0)
    int_sum_clamped = np.clip(int_sum, 0.0, 1.0)
    
    # Модельная масса: M = M0 - (M0 - Mfin) * α
    M0 = exp_mass[0]
    Mfin = exp_mass[-1]
    model_mass = M0 - (M0 - Mfin) * int_sum_clamped
    model_mass = np.clip(model_mass, Mfin, M0)
    
    # MSE
    mse = np.mean((model_mass - exp_mass) ** 2)
    
    return mse, sol

# === ТЕСТ ИНТЕГРАЦИИ ===
# Формируем вектор параметров из схемы
params_initial = np.concatenate([
    [rxn["log_A"] for rxn in reactions],           # logA
    [rxn["Ea"] for rxn in reactions],              # Ea (кДж/моль)
    [0 for _ in reactions],                         # model_idx (0 = первая модель в allowed_models)
    [rxn["contribution"] for rxn in reactions],    # contributions
])

print("=== Тест ОДУ интеграции ===")
print(f"Вектор параметров: {params_initial}")

for i, beta in enumerate(betas):
    try:
        mse, sol = integrate_ode_for_beta(
            beta, params_initial, species_list, reactions,
            num_species, num_reactions, exp_temperature, all_exp_masses[i]
        )
        status = "✓ OK" if sol is not None else "✗ Failed"
        print(f"  β={beta}: MSE={mse:.6f}, {status}")
        if sol is not None:
            print(f"         Конечные концентрации: {sol.y[:num_species, -1]}")
    except TimeoutError as e:
        print(f"  β={beta}: TIMEOUT - {e}")

print("\nОДУ функции готовы к использованию.")

=== Тест ОДУ интеграции ===
Вектор параметров: [  8.    8.    8.  120.  120.  120.    0.    0.    0.    0.5   0.5   0.5]
  β=3.0: MSE=0.003133, ✓ OK
         Конечные концентрации: [-2.03016635e-07 -1.50119647e-07 -2.79877587e-09  1.00000036e+00]
  β=5.0: MSE=0.005559, ✓ OK
         Конечные концентрации: [-2.04983964e-08 -3.86770946e-07 -2.63779449e-07  1.00000067e+00]
  β=10.0: MSE=0.006908, ✓ OK
         Конечные концентрации: [-5.00018338e-09 -2.56368395e-07 -6.98282786e-08  1.00000033e+00]

ОДУ функции готовы к использованию.


---

## Оптимизация параметров (differential_evolution)

### Целевая функция

Для минимизации MSE между моделью и экспериментом используется **дифференциальная эволюция** (DE) из SciPy:

$$ \text{MSE}_{\text{total}} = \sum_{\beta} \text{mean}\left((M_{\text{exp}}^{(\beta)} - M_{\text{model}}^{(\beta)})^2\right) $$

### Вектор параметров

Для схемы из $M$ реакций вектор оптимизации имеет структуру:

```
params = [logA₁, ..., logA_M, Ea₁, ..., Ea_M, model_idx₁, ..., model_idx_M, contrib₁, ..., contrib_M]
         └──── 0:M-1 ────┘    └─── M:2M-1 ───┘   └──── 2M:3M-1 ─────┘      └──── 3M:4M-1 ─────┘
```

### Параметры DE

| Параметр       | Значение  | Описание                           |
|----------------|-----------|-------------------------------------|
| `strategy`     | best1bin  | Стратегия мутации                   |
| `maxiter`      | 1000      | Макс. число итераций                |
| `popsize`      | 50        | Размер популяции                    |
| `tol`          | 0.01      | Допуск сходимости                   |
| `workers`      | 6         | Параллельные воркеры                |
| `init`         | latinhypercube | Начальная инициализация        |

### Timeout

Общий timeout расчёта — 2 минуты. Если оптимизация не сходится, возвращается лучший найденный результат.

---

## Профилирование производительности

Для оптимизации расчётов важно понимать, какие ресурсы потребляет алгоритм.

### Метрики

1. **Время выполнения** — общее время оптимизации
2. **Потребление памяти** — пиковое использование RAM
3. **Число итераций DE** — фактическое vs максимальное
4. **Число вызовов ОДУ** — сколько раз интегрировалась система

In [3]:
# %% [code]
import sys
import tracemalloc
import time
from scipy.optimize import differential_evolution

# === ГРАНИЦЫ ОПТИМИЗАЦИИ (bounds) ===
bounds = []
for rxn in reactions:
    bounds.append((rxn["log_A_min"], rxn["log_A_max"]))  # logA
for rxn in reactions:
    bounds.append((rxn["Ea_min"], rxn["Ea_max"]))  # Ea
for rxn in reactions:
    bounds.append((0, len(rxn["allowed_models"]) - 1))  # model_idx
for rxn in reactions:
    bounds.append((rxn["contribution_min"], rxn["contribution_max"]))  # contribution

# === ЦЕЛЕВАЯ ФУНКЦИЯ ===
def objective_function(params):
    """Целевая функция для differential_evolution."""
    total_mse = 0.0
    for beta, exp_mass in zip(betas, all_exp_masses):
        try:
            mse, _ = integrate_ode_for_beta(
                beta, params, species_list, reactions,
                num_species, num_reactions, exp_temperature, exp_mass
            )
            total_mse += mse
        except TimeoutError:
            return 1e4
    return total_mse

# === СЧЁТЧИК ВЫЗОВОВ ОДУ ===
ode_call_counter = 0

def objective_function_profiled(params):
    """Профилируемая версия целевой функции с подсчётом вызовов ОДУ."""
    global ode_call_counter
    total_mse = 0.0
    
    for beta, exp_mass in zip(betas, all_exp_masses):
        try:
            ode_call_counter += 1
            mse, _ = integrate_ode_for_beta(
                beta, params, species_list, reactions,
                num_species, num_reactions, exp_temperature, exp_mass
            )
            total_mse += mse
        except TimeoutError:
            return 1e4
    
    return total_mse

# === ПРОФИЛИРОВАНИЕ ОПТИМИЗАЦИИ ===
print("=== Профилирование производительности ===\n")

# Параметры для профилирования
profile_maxiter = 5
profile_popsize = 10

# Сброс счётчика
ode_call_counter = 0

# Запуск трассировки памяти
tracemalloc.start()

# Замер времени
profile_start = time.time()

# Запуск оптимизации с профилированием
profile_result = differential_evolution(
    objective_function_profiled,
    bounds,
    strategy=de_params["strategy"],
    maxiter=profile_maxiter,
    popsize=profile_popsize,
    tol=de_params["tol"],
    mutation=de_params["mutation"],
    recombination=de_params["recombination"],
    init=de_params["init"],
    workers=1,
    polish=False,
    updating="deferred",
    disp=False,
    x0=params_initial
)

profile_elapsed = time.time() - profile_start

# Получение статистики памяти
current_mem, peak_mem = tracemalloc.get_traced_memory()
tracemalloc.stop()

# === ВЫВОД МЕТРИК ===
print(f"{'='*50}")
print("=== МЕТРИКИ ПРОИЗВОДИТЕЛЬНОСТИ ===")
print(f"{'='*50}")

print(f"\n⏱️ ВРЕМЯ ВЫПОЛНЕНИЯ:")
print(f"   Профилируемый запуск ({profile_maxiter} итераций): {profile_elapsed:.2f} сек")
print(f"   Оценка для 1000 итераций: ~{profile_elapsed * 200:.0f} сек")

print(f"\n💾 ПАМЯТЬ:")
print(f"   Текущее потребление: {current_mem / 1024 / 1024:.2f} MB")
print(f"   Пиковое потребление: {peak_mem / 1024 / 1024:.2f} MB")

print(f"\n🔄 ИТЕРАЦИИ DE:")
print(f"   Выполнено итераций: {profile_result.nit} / {profile_maxiter} (maxiter)")
print(f"   Вычислений функции: {profile_result.nfev}")
print(f"   Лучшее MSE: {profile_result.fun:.6f}")

print(f"\n📊 ВЫЗОВЫ ОДУ:")
print(f"   За профилируемый запуск: {ode_call_counter}")
print(f"   На одну итерацию DE: ~{ode_call_counter // profile_maxiter if ode_call_counter > 0 else 0}")
print(f"   ОДУ вызовов на оценку MSE: {len(betas)} (по одному на каждый β)")

print(f"\n📈 СКОРОСТЬ ИНТЕГРИРОВАНИЯ:")
avg_ode_time = (profile_elapsed / ode_call_counter * 1000) if ode_call_counter > 0 else 0
print(f"   Среднее время одного ОДУ: {avg_ode_time:.1f} мс")
print(f"   Timeout на ОДУ: 50 мс")

print(f"\n{'='*50}")
print("✓ Профилирование завершено")

=== Профилирование производительности ===

=== МЕТРИКИ ПРОИЗВОДИТЕЛЬНОСТИ ===

⏱️ ВРЕМЯ ВЫПОЛНЕНИЯ:
   Профилируемый запуск (5 итераций): 102.84 сек
   Оценка для 1000 итераций: ~20568 сек

💾 ПАМЯТЬ:
   Текущее потребление: 1.09 MB
   Пиковое потребление: 1.47 MB

🔄 ИТЕРАЦИИ DE:
   Выполнено итераций: 5 / 5 (maxiter)
   Вычислений функции: 720
   Лучшее MSE: 0.584289

📊 ВЫЗОВЫ ОДУ:
   За профилируемый запуск: 1061
   На одну итерацию DE: ~212
   ОДУ вызовов на оценку MSE: 3 (по одному на каждый β)

📈 СКОРОСТЬ ИНТЕГРИРОВАНИЯ:
   Среднее время одного ОДУ: 96.9 мс
   Timeout на ОДУ: 50 мс

✓ Профилирование завершено
